In [24]:
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(usecwd=True), override=True)

import os
print("OPENAI ok:", bool(os.getenv("OPENAI_API_KEY")))
print("REPLICATE ok:", bool(os.getenv("REPLICATE_API_TOKEN")))

OPENAI ok: True
REPLICATE ok: True


In [26]:
# 기본 임포트
from typing import List, Dict, Literal, TypedDict
from pydantic import BaseModel, Field
from typing_extensions import Annotated

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END

In [ ]:
# 1) 스키마 정의 (EmotionResult / MusicBrief / GraphState)
Valence = Annotated[float, Field(ge=-1.0, le=1.0)]
Arousal = Annotated[float, Field(ge=0.0, le=1.0)]
BPM     = Annotated[int,   Field(ge=50,  le=140)]
DurSec  = Annotated[int,   Field(ge=30,   le=90)] # 음악 재생시간(30초 ~ 90초)

class EmotionResult(BaseModel):
    primary: str
    valence: Valence = 0.0
    arousal: Arousal = 0.5
    confidence: Arousal = 0.7
    reasons: str = "—"

class MusicBrief(BaseModel):
    mood: str
    bpm: BPM = 90
    key: str = "C major"
    duration_sec: DurSec = 60               # 음악 기본 재생시간 60초
    instruments: List[str] = []
    style_tags: List[str] = []
    prompt: str  # 영어 프롬프트

class GraphState(TypedDict, total=False):
    user_text: str
    emotion: EmotionResult
    brief: MusicBrief
    audio_path: str
    provider_used: Literal["replicate"]
    meta: Dict


In [28]:
# 2) LLM 헬퍼 (OpenAI)
def get_llm():
    # 스키마 맞춤 출력이 중요 → 낮은 temperature
    return ChatOpenAI(model="gpt-4o-mini", temperature=0.2)


In [29]:
# 3) 감정 분석 노드
def analyze_emotion_node(state: GraphState) -> GraphState:
    llm = get_llm()
    sys = (
        "당신은 심리 정서를 요약하는 분석가입니다. "
        "사용자 텍스트에서 주요 감정을 한 단어(또는 짧은 구)로 도출하고, "
        "valence(-1~1), arousal(0~1), confidence(0~1)을 추정하세요. "
        "반드시 EmotionResult(JSON 스키마)에 맞춰 응답하세요."
    )
    structured = llm.with_structured_output(EmotionResult)
    result = structured.invoke([
        {"role":"system","content":sys},
        {"role":"user","content":state["user_text"]}
    ])
    state["emotion"] = result
    return state

In [ ]:
# 4) 음악 브리프 노드
def compose_brief_node(state: GraphState) -> GraphState:
    llm = get_llm()
    emo: EmotionResult = state["emotion"]
    sys = (
        "너는 음악 감독이다. 아래 감정 분석과 사용자 텍스트를 참고해 "
        "치유/안정 목적의 짧은 BGM을 위한 Music Brief를 JSON으로 만들어라. "
        "bpm=50~140, duration_sec=30~90, key는 'C major' 같은 형식. "
        "prompt는 영어로 핵심 악기/무드/다이내믹을 간결히 포함."
    )
    usr = (
        f"# Emotion\nprimary={emo.primary}, valence={emo.valence}, "
        f"arousal={emo.arousal}, confidence={emo.confidence}\n\n"
        f"# Text\n{state['user_text']}\n"
    )
    structured = llm.with_structured_output(MusicBrief)
    brief = structured.invoke([
        {"role":"system","content":sys},
        {"role":"user","content":usr}
    ])

# ▼ duration 보정 (30~90초로 강제)
    if brief.duration_sec < 30:
        brief = brief.model_copy(update={"duration_sec": 30})
    elif brief.duration_sec > 90:
        brief = brief.model_copy(update={"duration_sec": 90})

    state["brief"] = brief
    return state

In [ ]:
# 5) Replicate(Stable Audio 2.5) 호출 함수
import time, tempfile, requests, soundfile as sf, os, numpy as np
import replicate

# 모델 버전은 바뀔 수 있음 — 404 나오면 Replicate 모델 페이지에서 최신 버전ID로 교체
MODEL_ID = "stability-ai/stable-audio-2.5:46a2601577d0e31aa99b03c9d7fd2142fa3b96a282338758f794b620e35c75b7"

def generate_with_replicate_strict(prompt: str, duration: int) -> str:
    tok = os.getenv("REPLICATE_API_TOKEN")
    assert tok, "REPLICATE_API_TOKEN이 없습니다 (.env 확인)"
    assert 30 <= int(duration) <= 90, f"duration(초)은 30~90 범위여야 합니다: {duration}"

    # 1) 모델 실행
    urls = replicate.run(MODEL_ID, input={"prompt": prompt, "duration": int(duration)})
    if not (isinstance(urls, list) and urls):
        raise RuntimeError(f"Replicate 결과 비어있음: {urls}")

    # 2) 결과 파일 다운로드 → 포맷 자동 감지 후 모노 WAV 저장
    url = urls[0]
    r = requests.get(url, timeout=120); r.raise_for_status()

    with tempfile.NamedTemporaryFile(suffix=".bin", delete=True) as tmp:
        tmp.write(r.content); tmp.flush()
        data, sr = sf.read(tmp.name)     # mp3/wav 자동
        if data.ndim > 1:
            data = data.mean(axis=1).astype("float32")

    os.makedirs("outputs", exist_ok=True)
    out_path = f"outputs/stableaudio_{int(time.time())}.wav"
    sf.write(out_path, data, sr)
    return out_path


In [32]:
# 6) 음악 생성 노드 (Replicate만)
def generate_music_node(state: GraphState) -> GraphState:
    brief: MusicBrief = state["brief"]
    path = generate_with_replicate_strict(brief.prompt, int(brief.duration_sec))

    state["audio_path"] = path
    state["provider_used"] = "replicate"
    state["meta"] = {
        "emotion": state["emotion"].model_dump(),
        "brief": state["brief"].model_dump(),
        "provider": "replicate",
        "path": path,
    }
    return state

In [33]:
# 임시) 분기 함수 추가 (credit문제로 인해
def should_generate(state: GraphState) -> str:
    # DRY_RUN=1 이거나 REPLICATE_API_TOKEN 없으면 건너뜀
    dry = os.getenv("DRY_RUN", "0") == "1"
    has_repl = bool(os.getenv("REPLICATE_API_TOKEN"))
    return "go" if (not dry and has_repl) else "skip"

# 임시
os.environ["DRY_RUN"] = "1"
print("DRY_RUN:", os.getenv("DRY_RUN"))

DRY_RUN: 1


In [34]:
workflow = StateGraph(GraphState)
workflow.add_node("analyze_emotion", analyze_emotion_node)
workflow.add_node("compose_brief",  compose_brief_node)
workflow.add_node("generate_music", generate_music_node)

workflow.add_edge(START, "analyze_emotion")
workflow.add_edge("analyze_emotion", "compose_brief")
# workflow.add_edge("compose_brief",  "generate_music")
# workflow.add_edge("generate_music", END)

#임시
workflow.add_conditional_edges(
    "compose_brief",
    should_generate,
    {"go": "generate_music", "skip": END}
)
workflow.add_edge("generate_music", END)


graph = workflow.compile()

In [ ]:
state = {
    "user_text": "오늘 중요한 발표를 앞두고 불안하지만 차분해지고 싶어요."
}
final = graph.invoke(state)

print("=== Emotion ===")
print(final["emotion"].model_dump())
print("\n=== Music Brief ===")
print(final["brief"].model_dump())
# print("\n=== Provider Used ===")
# print(final["provider_used"])
# print("\n=== Audio Path ===")
# print(final["audio_path"])


=== Emotion ===
{'primary': '불안', 'valence': -0.5, 'arousal': 0.7, 'confidence': 0.8, 'reasons': '발표를 앞두고 불안함을 느끼고 있지만, 차분해지고 싶다는 의지가 나타나고 있어 감정의 복합성이 존재함.'}

=== Music Brief ===
{'mood': 'Calm', 'bpm': 70, 'key': 'C major', 'duration_sec': 30, 'instruments': ['Piano', 'Strings', 'Flute'], 'style_tags': ['Ambient', 'Relaxing', 'Meditative'], 'prompt': 'A soothing piano melody with soft strings and gentle flute, creating a calm atmosphere to ease anxiety before a presentation.'}

=== Provider Used ===


KeyError: 'provider_used'